In [209]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Colab/dataset/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Colab/dataset/test.csv')

In [212]:
def num_null(df):
    missing = df.isnull().sum()
    print('Show #missing in the columns:')
    for i in range(df.shape[1]):
        if missing[i]:
            print(missing.index[i], ':', missing[i])
            

def metric(truth, pred):
    truth = np.array(truth)
    pred = np.array(pred)
    diff = abs(pred - truth) / truth
    print(list(diff <= 0.1).count(True) / len(diff))
    

num_null(train)

Show #missing in the columns:
parking_area : 56897
parking_price : 46065
txn_floor : 15902
village_income_median : 1142


In [0]:
train['total_price'] = np.log1p(train["total_price"] / train["building_area"])

In [0]:
X = pd.concat([train.drop(['total_price'], axis=1), test])
X.reset_index(inplace=True, drop=True)

#### Imputation

missing value imputation strategy:   
parking_area：補零  
parking_price：中位數  
txn_floor：中位數  
village_income_median：以經緯度取kNN

In [215]:
num_null(X)

Show #missing in the columns:
parking_area : 66397
parking_price : 53775
txn_floor : 18541
village_income_median : 1326


In [0]:
pd.options.mode.chained_assignment = None

In [0]:
X_imp = X.copy()
X_imp['parking_area'].fillna(0, inplace=True)
X_imp['parking_price'].fillna(X_imp['parking_price'].median(), inplace=True)
# X_imp['txn_floor'].fillna(X_imp['txn_floor'].median(), inplace=True)
X_imp['txn_floor'].fillna(-1, inplace=True)
# X_imp['txn_floor'].fillna(X_imp['total_floor'] / 2, inplace=True)

In [0]:
# knn_impute(X_imp, 'village_income_median', ['lat', 'lon'])
vimm = X_imp.groupby('village').agg({'village_income_median':'mean'})

vim_isnan = np.isnan(X_imp['village_income_median'])
for i in range(len(X_imp)):
    if vim_isnan[i]:
        X_imp['village_income_median'][i] = vimm.loc[X_imp['village'][i], 'village_income_median']

In [219]:
num_null(X_imp)

Show #missing in the columns:
village_income_median : 938


In [0]:
tvimm = X_imp.groupby('town').agg({'village_income_median':'median'})

vim_isnan = np.isnan(X_imp['village_income_median'])
for i in range(len(X_imp)):
    if vim_isnan[i]:
        X_imp['village_income_median'][i] = tvimm.loc[X_imp['town'][i], 'village_income_median']

# knn_impute(X_imp, 'village_income_median', ['lat', 'lon'])

In [221]:
num_null(X_imp)

Show #missing in the columns:


In [0]:
# X_imp['dt_diff'] = X_imp['txn_dt'] - X_imp['building_complete_dt']

In [0]:
# X_imp.drop(list(X_imp.filter(regex=('_MIN'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('_index_'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('^N'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('_10$'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('_50$'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('_100$'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('_250$'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('_500$'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('_1000$'))), axis=1, inplace=True)
# print(X_imp.shape)
# X_imp.drop(list(X_imp.filter(regex=('_5000$'))), axis=1, inplace=True)
# print(X_imp.shape)

In [0]:
# concat_train = pd.concat([X_train, Y_train], axis=1)
concat_train = train.copy()

# cat_mean_df = concat_train.groupby('building_material').agg({'total_price':'mean'})
# X_imp['building_material'] = list(cat_mean_df.loc[list(X_imp['building_material']), :]['total_price'])

# cat_mean_df = concat_train.groupby('city').agg({'total_price':'mean'})
# X_imp['city'] = list(cat_mean_df.loc[list(X_imp['city']), :]['total_price'])

# cat_mean_df = concat_train.groupby('town').agg({'total_price':'mean'})
# X_imp['town'] = list(cat_mean_df.loc[list(X_imp['town']), :]['total_price'])

# cat_mean_df = concat_train.groupby('village').agg({'total_price':'mean'})
# X_imp['village'] = list(cat_mean_df.loc[list(X_imp['village']), :]['total_price'])

# cat_mean_df = concat_train.groupby('building_type').agg({'total_price':'mean'})
# X_imp['building_type'] = list(cat_mean_df.loc[list(X_imp['building_type']), :]['total_price'])

# cat_mean_df = concat_train.groupby('building_use').agg({'total_price':'mean'})
# X_imp['building_use'] = list(cat_mean_df.loc[list(X_imp['building_use']), :]['total_price'])

# cat_mean_df = concat_train.groupby('parking_way').agg({'total_price':'mean'})
# X_imp['parking_way'] = list(cat_mean_df.loc[list(X_imp['parking_way']), :]['total_price'])





building_material_dummies = pd.get_dummies(X_imp['building_material'])
building_material_dummies.columns = ['building_material_' + str(col) for col in building_material_dummies.columns]
X_imp = pd.concat([X_imp, building_material_dummies], axis=1)
X_imp.drop(['building_material', building_material_dummies.columns[0]], inplace=True, axis=1)

city_dummies = pd.get_dummies(X_imp['city'])
city_dummies.columns = ['city_' + str(col) for col in city_dummies.columns]
X_imp = pd.concat([X_imp, city_dummies], axis=1)
X_imp.drop(['city', city_dummies.columns[0]], inplace=True, axis=1)

# town_dummies = pd.get_dummies(X_imp['town'])
# town_dummies.columns = ['town_' + str(col) for col in town_dummies.columns]
# X_imp = pd.concat([X_imp, town_dummies], axis=1)
# X_imp.drop(['town', town_dummies.columns[0]], inplace=True, axis=1)

# village_dummies = pd.get_dummies(X_imp['village'])
# village_dummies.columns = ['village_' + str(col) for col in village_dummies.columns]
# X_imp = pd.concat([X_imp, village_dummies], axis=1)
# X_imp.drop(['village', village_dummies.columns[0]], inplace=True, axis=1)

building_type_dummies = pd.get_dummies(X_imp['building_type'])
building_type_dummies.columns = ['building_type_' + str(col) for col in building_type_dummies.columns]
X_imp = pd.concat([X_imp, building_type_dummies], axis=1)
X_imp.drop(['building_type', building_type_dummies.columns[0]], inplace=True, axis=1)

building_use_dummies = pd.get_dummies(X_imp['building_use'])
building_use_dummies.columns = ['building_use_' + str(col) for col in building_use_dummies.columns]
X_imp = pd.concat([X_imp, building_use_dummies], axis=1)
X_imp.drop(['building_use', building_use_dummies.columns[0]], inplace=True, axis=1)

parking_way_dummies = pd.get_dummies(X_imp['parking_way'])
parking_way_dummies.columns = ['parking_way_' + str(col) for col in parking_way_dummies.columns]
X_imp = pd.concat([X_imp, parking_way_dummies], axis=1)
X_imp.drop(['parking_way', parking_way_dummies.columns[0]], inplace=True, axis=1)


In [0]:
# X_imp.drop(['parking_price', 'parking_area', 'parking_way'], axis=1, inplace = True)

In [0]:
post_train = X_imp.head(60000)
X_test = X_imp.tail(10000)

post_train.drop('building_id', axis=1, inplace=True)
post_train['total_price'] = train.head(60000)['total_price']

test_building_id = X_test['building_id']
X_test.drop('building_id', axis=1, inplace=True)

In [0]:
# rm_outlier_cols = ['total_price']
# for col in rm_outlier_cols:
#     col99 = np.percentile(post_train[col], 99)
# #     col01 = np.percentile(post_train[col], 1)
#     post_train = post_train[post_train[col] <= col99]
# #     post_train = post_train[post_train[col] >= col01]

In [0]:
# from sklearn.model_selection import train_test_split
# X_train_valid, X_test, Y_train_valid, Y_test = train_test_split(
#                         post_train.drop('total_price', axis=1), post_train['total_price'], test_size=0.2, random_state=42)
# X_train, X_valid, Y_train, Y_valid = train_test_split(
#                         X_train_valid, Y_train_valid, test_size=0.15, random_state=41)


In [0]:
# from xgboost import XGBRegressor
# from xgboost import plot_importance
# import time

# ts = time.time()

# model = XGBRegressor(
#     max_depth=16,
#     n_estimators=10000,
#     learning_rate=0.01,
#     nthread=8)

# model.fit(
#     X_train, 
#     Y_train,
#     eval_metric="rmse", 
#     eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
#     verbose=False,
#     early_stopping_rounds=20)

# print(time.time() - ts)


# Y_valid_predict = model.predict(X_valid)
# Y_valid_predict = np.floor(np.expm1(Y_valid_predict))
# Y_valid = np.floor(np.expm1(Y_valid))
# metric(Y_valid, Y_valid_predict)

In [0]:
# Y_test_predict = model.predict(X_test)
# Y_test_predict = np.floor(np.expm1(Y_test_predict)) * X_test['building_area']

In [0]:
# submission = pd.DataFrame({
#     "building_id": test_building_id, 
#     "total_price": Y_test_predict
# })
# submission.to_csv('./submission/submission_area1.csv', index=False)

#### train valid (all, no self test set)

In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(
                        post_train.drop('total_price', axis=1), post_train['total_price'], test_size=5000/60000, random_state=42)

In [232]:
ts = time.time()

model = XGBRegressor(
    max_depth=16,
    n_estimators=10000,
    learning_rate=0.01,
    nthread=8)

model.fit(
    X_train, 
    Y_train,
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
    verbose=False,
    early_stopping_rounds=20)

print(time.time() - ts)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[07:51:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
5059.769819974899


In [0]:
Y_test_predict = model.predict(X_test)
Y_test_predict = np.floor(np.expm1(Y_test_predict)) * X_test['building_area']

In [0]:
submission = pd.DataFrame({
    "building_id": test_building_id, 
    "total_price": Y_test_predict
})
submission.to_csv('/content/gdrive/My Drive/Colab/output/submission_xgb_area_valid.csv', index=False)